In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None


In [2]:
# Parameters
product = {"nb": "/home/luuka/thesis/workspace/output/02_train.ipynb", "model": "/home/luuka/thesis/workspace/checkpoints/best_model.pt", "metrics": "/home/luuka/thesis/workspace/checkpoints/best_model_info.json", "results": "/home/luuka/thesis/workspace/checkpoints/training_results.json"}


In [3]:
# Imports and Setup
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import logging
import json
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Add project root to path
sys.path.append(str(Path.cwd().parent))

# Import custom modules
from src.models.bert_model import TrollDetector
from src.models.trainer import TrollDetectorTrainer
from src.data_tools.dataset import TrollDataset, collate_batch

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Define paths
DATA_DIR = Path('data')
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
CHECKPOINT_DIR = Path('checkpoints')

# Create checkpoint directory
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

# Training configuration
config = {
    'model_name': 'distilbert-base-multilingual-cased',
    'max_length': 96,
    'batch_size': 64,
    'learning_rate': 2e-5,
    'weight_decay': 0.03,
    'num_epochs': 3,
    'dropout_rate': 0.2,
    'warmup_steps': 50,
    'max_grad_norm': 1.0,
    'comments_per_user': 5,
    'early_stopping_patience': 3,
    'use_wandb': False,
    'random_state': 17,  # Default if config not found
    'label_smoothing': 0.1        # Added label smoothing
}

# Try to load preprocessing config
try:
    with open(PROCESSED_DATA_DIR / 'preprocessing_config.json', 'r') as f:
        preproc_config = json.load(f)
        config['random_state'] = preproc_config.get('random_state', 42)
except FileNotFoundError:
    print("Warning: preprocessing_config.json not found, using default random_state")

print("Configuration loaded:")
for key, value in config.items():
    print(f"{key}: {value}")

Configuration loaded:
model_name: distilbert-base-multilingual-cased
max_length: 96
batch_size: 64
learning_rate: 2e-05
weight_decay: 0.03
num_epochs: 3
dropout_rate: 0.2
warmup_steps: 50
max_grad_norm: 1.0
comments_per_user: 5
early_stopping_patience: 3
use_wandb: False
random_state: 42
label_smoothing: 0.1


In [5]:
# Load preprocessed data splits
train_df = pd.read_parquet(PROCESSED_DATA_DIR / 'train.parquet')
val_df = pd.read_parquet(PROCESSED_DATA_DIR / 'val.parquet')
test_df = pd.read_parquet(PROCESSED_DATA_DIR / 'test.parquet')

print("Dataset sizes:")
print(f"Train: {len(train_df)} samples, {train_df['author'].nunique()} authors")
print(f"Val:   {len(val_df)} samples, {val_df['author'].nunique()} authors")
print(f"Test:  {len(test_df)} samples, {test_df['author'].nunique()} authors")

Dataset sizes:
Train: 2407195 samples, 17033 authors
Val:   541073 samples, 3650 authors
Test:  561855 samples, 3651 authors


In [6]:
# Create Datasets and DataLoaders
# Initialize datasets
train_dataset = TrollDataset(
    train_df,
    tokenizer_name=config['model_name'],
    max_length=config['max_length'],
    comments_per_user=config['comments_per_user']
)

val_dataset = TrollDataset(
    val_df,
    tokenizer_name=config['model_name'],
    max_length=config['max_length'],
    comments_per_user=config['comments_per_user']
)

test_dataset = TrollDataset(
    test_df,
    tokenizer_name=config['model_name'],
    max_length=config['max_length'],
    comments_per_user=config['comments_per_user']
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    collate_fn=collate_batch
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    collate_fn=collate_batch
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    collate_fn=collate_batch
)

INFO:src.data_tools.dataset:Created 152861 samples from 17033 authors


INFO:src.data_tools.dataset:Created 32204 samples from 3650 authors


INFO:src.data_tools.dataset:Created 31401 samples from 3651 authors


In [7]:
# Initialize Model and Trainer
model = TrollDetector(
    model_name=config['model_name'],
    dropout_rate=config['dropout_rate']
)

# Initialize trainer
trainer = TrollDetectorTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    learning_rate=config['learning_rate'],
    weight_decay=config['weight_decay'],
    max_grad_norm=config['max_grad_norm'],
    num_epochs=config['num_epochs'],
    warmup_steps=config['warmup_steps'],
    checkpoint_dir=CHECKPOINT_DIR,
    use_wandb=config['use_wandb']
)

/home/luuka/thesis/workspace/src/models/trainer.py:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()


In [8]:
# Train the model
final_metrics = trainer.train()

print("\nTraining completed!")
print("\nFinal metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save final configuration and results
results = {
    'config': config,
    'final_metrics': final_metrics
}

with open(CHECKPOINT_DIR / 'training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

INFO:src.models.trainer:Starting training on device: cuda


INFO:src.models.trainer:Training samples: 152861


INFO:src.models.trainer:Validation samples: 32204


INFO:src.models.trainer:
Epoch 1/3


Training:   0%|                                                                                | 0/2389 [00:00<?, ?it/s]

...

Evaluating:   5%|███▊                                                                  | 27/504 [00:33<08:12,  1.03s/it]

Evaluating:   6%|███▉                                                                  | 28/504 [00:34<09:16,  1.17s/it]

Evaluating:   6%|████                                                                  | 29/504 [00:36<10:03,  1.27s/it]

Evaluating:   6%|████▏                                                                 | 30/504 [00:38<10:45,  1.36s/it]

Evaluating:   6%|████▎                                                                 | 31/504 [00:39<11:15,  1.43s/it]